In [ ]:
# MPI的基础方法
# 导入MPI和辅助函数
from mpi4py import MPI  # MPI主模块
from mpi4py.MPI import COMM_WORLD  # COMM_WORLD通信器
from types import FunctionType  # 用于实现MPI进程池


class Pool(object):  # MPI进程池
    def __init__(self):
        self.f = None  # 对执行函数的引用，进程池开始时没有函数
        self.P = COMM_WORLD.Get_size()  # 处理器的总数
        self.rank = COMM_WORLD.Get_rank()  # 所位于的处理器


    def wait(self):  # 当进程池没有任务时接收数据的方法
        if self.rank == 0:  # 主进程不能等待
            raise RuntimeError("Proc 0 cannot wait!")
        status = MPI.Status()
        while True:
            task = COMM_WORLD.recv(source=0, tag=MPI.ANY_TAG, status=status)  # 从主进程接收新任务
            if not task:
                break  # 无新任务，不再接收
            if isinstance(task, FunctionType):  # 如果任务是一个函数，将其放在对象上，然后继续等待
                self.f = task
                continue
            result = self.f(task)  # 调用此任务上的函数并发送回结果
            COMM_WORLD.isend(result, dest=0, tag=status.tag)


    def map(self, f, tasks):  # map()方法
        N = len(tasks)  # 任务数
        P = self.P  # 处理器数
        Pless1 = P - 1  # 留一个处理器用于调度
        if self.rank != 0:  # 让worker等待主进程发出任务
            self.wait()
            return

        if f is not self.f:
            self.f = f
            requests = []
            for p in range(1, self.P):  # 发送所有worker的函数
                r = COMM_WORLD.isend(f, dest=p)
                requests.append(r)
            MPI.Request.waitall(requests)

        requests = []
        for i, task in enumerate(tasks):  # 将任务平均分配给所有worker
            r = COMM_WORLD.isend(task, dest=(i%Pless1)+1, tag=i)
            requests.append(r)
        MPI.Request.waitall(requests)

        results = []
        for i in range(N):  # 等待worker返回结果
            result = COMM_WORLD.recv(source=(i%Pless1)+1, tag=i)
            results.append(result)
        return results


    def __del__(self):
        if self.rank == 0:
            for p in range(1, self.P):  # 当进程池关闭时，关闭所有worker
                COMM_WORLD.isend(False, dest=p)

函数自行调度，map()为主要应用方法。